In [202]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import boto3
import io
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

In [203]:
# ───────────── Settings ─────────────
BUCKET       = 'bdc-public-curated'
PREFIX       = 'ndacan/nytd/outcomes/waves_processed/'

In [204]:
# ID and metadata columns
ID_COLS   = ['StFIPS','St', 'RepDate',]
DEM_COLS  = ['Sex','Race'] 

# Potential Independents (OR‐combined across Wave 2 & 3)
INDEPENDENTS = [
    'HighEdCert','CurrenRoll','CurrFTE','CurrPTE','Homeless','SubAbuse','Incarc'
]

In [205]:
# ───────────── Initialize S3 client & list files ─────────────
s3 = boto3.client('s3')

# Helpers

In [206]:
def s3_read_csv(key):
    """Load a CSV from S3 into a pandas DataFrame."""
    obj = s3.get_object(Bucket=BUCKET, Key=key)
    return pd.read_csv(io.BytesIO(obj['Body'].read()), dtype=str)

In [207]:
def normalize_keys(df, wave_tag):
    # 1) rename StFIPS_wX → StFIPS, etc.
    for col in ID_COLS:
        suff = f"{col}_{wave_tag}"
        if suff in df.columns:
            df.rename(columns={suff: col}, inplace=True)
    # 2) trim whitespace & unify types
    df[ID_COLS] = df[ID_COLS].astype(str).apply(lambda s: s.str.strip())
    # 3) parse dates
    df['RepDate'] = pd.to_datetime(df['RepDate'], errors='coerce')
    return df

In [208]:
# ───────────── 1) LOAD & NORMALIZE WAVES ─────────────
wave1 = s3_read_csv(f"{PREFIX}wave1.csv")
wave2 = s3_read_csv(f"{PREFIX}wave2.csv")
wave3 = s3_read_csv(f"{PREFIX}wave3.csv")
all_wave = s3_read_csv(f"{PREFIX}cleaned_all_waves.csv")
wave1 = normalize_keys(wave1, "w1")
wave2 = normalize_keys(wave2, "w2")
wave3 = normalize_keys(wave3, "w3")

print("Wave shapes:", wave1.shape, wave2.shape, wave3.shape)


Wave shapes: (3068, 38) (1362, 38) (950, 38)


### Wave 1

In [209]:
# 2) Drop any column suffixed with '_w2' or '_w3'
wave1 = all_wave.drop(
    columns=[c for c in all_wave.columns if c.endswith(('_w23'))]
).copy()

# 3) Optionally strip off the '_w1' suffix so your columns are the “base” names
wave1.columns = [
    re.sub(r'_w1$', '', c)
    for c in wave1.columns
]

In [210]:
# print the (rows, columns) tuple
print(wave1.shape)

(720, 42)


In [211]:
# 2) Drop any column suffixed with '_w2' or '_w3'
wave23 = all_wave.drop(
    columns=[c for c in all_wave.columns if c.endswith(('_w1'))]
).copy()

# 3) Optionally strip off the '_w1' suffix so your columns are the “base” names
wave23.columns = [
    re.sub(r'_w1$', '', c)
    for c in wave23.columns
]

In [212]:
# print the (rows, columns) tuple
print(wave23.shape)

(720, 41)


In [213]:
print(f"Wave1 shape: {wave1.shape}")
print(f"Wave23 shape: {wave23.shape}")

Wave1 shape: (720, 42)
Wave23 shape: (720, 41)


# 1, 0 and % present of all

In [214]:
# Read the uploaded CSV file
df = pd.read_csv('all_waves_merged_final.csv')

print("=== SIMPLE 1s AND 0s COUNT FOR ALL COLUMNS ===")
print(f"Dataset shape: {df.shape}")
print(f"Total rows: {len(df)}")
print(f"Total columns: {len(df.columns)}")

# Initialize results list
results = []

# Go through each column and just count 1s and 0s - ignore everything else
for column in df.columns:
    col_data = df[column]
    
    # Count 1s and 0s (handle both int and float)
    count_1 = ((col_data == 1) | (col_data == 1.0)).sum()
    count_0 = ((col_data == 0) | (col_data == 0.0)).sum()
    
    # Count blanks/missing
    blank_count = col_data.isna().sum()
    
    # Total 1s + 0s
    ones_zeros_total = count_1 + count_0
    
    # Calculate percentages (only if we have any 1s or 0s)
    if ones_zeros_total > 0:
        percent_1 = (count_1 / ones_zeros_total) * 100
        percent_0 = (count_0 / ones_zeros_total) * 100
    else:
        percent_1 = 0
        percent_0 = 0
    
    # Store results
    results.append({
        'Column': column,
        'Total_Rows': len(col_data),
        'Count_0': int(count_0),
        'Count_1': int(count_1),
        'Blank_Count': int(blank_count),
        'Ones_Zeros_Total': int(ones_zeros_total),
        'Percent_0': round(percent_0, 2),
        'Percent_1': round(percent_1, 2)
    })

# Convert to DataFrame
results_df = pd.DataFrame(results)

print(f"\n=== ALL COLUMNS - 1s AND 0s COUNT ===")
display_cols = ['Column', 'Count_0', 'Count_1', 'Ones_Zeros_Total', 'Percent_0', 'Percent_1', 'Blank_Count']
print(results_df[display_cols].to_string(index=False))

# Show columns that actually have 1s or 0s
has_ones_or_zeros = results_df[results_df['Ones_Zeros_Total'] > 0]

print(f"\n=== COLUMNS WITH ANY 1s OR 0s ({len(has_ones_or_zeros)} columns) ===")
if len(has_ones_or_zeros) > 0:
    print(has_ones_or_zeros[display_cols].to_string(index=False))
    
    # Top columns by count of 1s
    print(f"\n=== TOP COLUMNS BY NUMBER OF 1s ===")
    top_by_ones = has_ones_or_zeros.nlargest(10, 'Count_1')
    for _, row in top_by_ones.iterrows():
        print(f"{row['Column']}: {row['Count_1']} ones, {row['Count_0']} zeros ({row['Percent_1']:.1f}% ones)")
else:
    print("No 1s or 0s found in any columns")

# Save results
results_df.to_csv('simple_ones_zeros_count.csv', index=False)
print(f"\nResults saved to: simple_ones_zeros_count.csv")

# Show some sample data to verify we're reading correctly
print(f"\n=== SAMPLE DATA VERIFICATION ===")
print("First few rows of first 5 columns:")
print(df.iloc[:5, :5])

print(f"\nData types of first 10 columns:")
for col in df.columns[:10]:
    print(f"{col}: {df[col].dtype}")

=== SIMPLE 1s AND 0s COUNT FOR ALL COLUMNS ===
Dataset shape: (720, 78)
Total rows: 720
Total columns: 78

=== ALL COLUMNS - 1s AND 0s COUNT ===
             Column  Count_0  Count_1  Ones_Zeros_Total  Percent_0  Percent_1  Blank_Count
             StFCID        0        0                 0       0.00       0.00            0
             StFIPS        0        0                 0       0.00       0.00          406
                 St        0        0                 0       0.00       0.00            0
         RepDate_w1        0        0                 0       0.00       0.00          720
                Sex        0      411               411       0.00     100.00            0
        OutcmRpt_w1        0      720               720       0.00     100.00            0
        OutcmDte_w1        0        0                 0       0.00       0.00          720
        OutcmFCS_w1        0      720               720       0.00     100.00            0
         CurrFTE_w1      702        

In [225]:
# Read the uploaded CSV file
df = pd.read_csv('all_waves_merged_final.csv')

print("=== WAVE1, WAVE23, AND COMBINED 1s/0s ANALYSIS ===")
print(f"Dataset shape: {df.shape}")

# Initialize results list
results = []

# Go through each column and count 1s and 0s
for column in df.columns:
    col_data = df[column]
    
    # Count 1s and 0s
    count_1 = ((col_data == 1) | (col_data == 1.0)).sum()
    count_0 = ((col_data == 0) | (col_data == 0.0)).sum()
    ones_zeros_total = count_1 + count_0
    
    # Calculate percent of 1s
    if ones_zeros_total > 0:
        percent_1 = (count_1 / ones_zeros_total) * 100
    else:
        percent_1 = 0
    
    # Store results
    results.append({
        'Column': column,
        'Count_0': int(count_0),
        'Count_1': int(count_1),
        'Percent_1': round(percent_1, 2)
    })

# Convert to DataFrame
results_df = pd.DataFrame(results)

# Filter only columns with any 1s or 0s
has_data = results_df[(results_df['Count_0'] > 0) | (results_df['Count_1'] > 0)]

# === WAVE1 COLUMNS ===
wave1_columns = has_data[has_data['Column'].str.endswith('_w1')]
print(f"\n=== WAVE1 COLUMNS ({len(wave1_columns)} columns) ===")
print("Column                Count_0  Count_1  Percent_1")
print("-" * 50)
for _, row in wave1_columns.iterrows():
    print(f"{row['Column']:<20} {row['Count_0']:>7} {row['Count_1']:>8} {row['Percent_1']:>9}%")

# === WAVE23 COLUMNS ===
wave23_columns = has_data[has_data['Column'].str.endswith('_w23')]
print(f"\n=== WAVE23 COLUMNS ({len(wave23_columns)} columns) ===")
print("Column                Count_0  Count_1  Percent_1")
print("-" * 50)
for _, row in wave23_columns.iterrows():
    print(f"{row['Column']:<20} {row['Count_0']:>7} {row['Count_1']:>8} {row['Percent_1']:>9}%")


# === SUMMARY TABLES ===
print(f"\n=== SUMMARY STATISTICS ===")
print("\nWAVE1 SUMMARY:")
print(f"  Total columns: {len(wave1_columns)}")
print(f"  Total 0s: {wave1_columns['Count_0'].sum()}")
print(f"  Total 1s: {wave1_columns['Count_1'].sum()}")
print(f"  Average % of 1s: {wave1_columns['Percent_1'].mean():.2f}%")

print("\nWAVE23 SUMMARY:")
print(f"  Total columns: {len(wave23_columns)}")
print(f"  Total 0s: {wave23_columns['Count_0'].sum()}")
print(f"  Total 1s: {wave23_columns['Count_1'].sum()}")
print(f"  Average % of 1s: {wave23_columns['Percent_1'].mean():.2f}%")




=== WAVE1, WAVE23, AND COMBINED 1s/0s ANALYSIS ===
Dataset shape: (720, 78)

=== WAVE1 COLUMNS (35 columns) ===
Column                Count_0  Count_1  Percent_1
--------------------------------------------------
OutcmRpt_w1                0      720     100.0%
OutcmFCS_w1                0      720     100.0%
CurrFTE_w1               702        8      1.13%
CurrPTE_w1               640       71      9.99%
EmplySklls_w1            588      119     16.83%
SocSecrty_w1             658       50      7.06%
EducAid_w1               636       13       2.0%
PubFinAs_w1              117        5       4.1%
PubFoodAs_w1             115        6      4.96%
PubHousAs_w1             119        2      1.65%
OthrFinAs_w1             689       25       3.5%
HighEdCert_w1              0       32     100.0%
CurrenRoll_w1             33      681     95.38%
CnctAdult_w1              34      685     95.27%
Homeless_w1              634       82     11.45%
SubAbuse_w1              447      271     37.74%
Inc

In [226]:
# show all column names
print(wave23.columns.tolist())

# and/or show a sample of the first few rows
print(wave23.head())


['StFCID', 'StFIPS', 'St', 'Sex', 'Cohort', 'PubFoodAs', 'SubAbuse', 'Assoc_Degree', 'Responded', 'HS_or_GED', 'Higher_Degree', 'RepDate', 'HighEdCert', 'Homeless', 'OthrHlthIn', 'PubHousAs', 'Baseline', 'Marriage', 'CurrFTE', 'PubFinAs', 'MentlHlthIn', 'Voc_License', 'OutcmFCS', 'CnctAdult', 'Voc_Certificate', 'CurrenRoll', 'OthrFinAs', 'SocSecrty', 'Medicaid', 'OutcmDte', 'Incarc', 'Elig21', 'CurrPTE', 'EducAid', 'MedicalIn', 'OutcmRpt', 'EmplySklls', 'Children', 'PrescripIn', 'Bach_Degree']
           StFCID StFIPS  St  Sex Cohort PubFoodAs SubAbuse Assoc_Degree  \
0  TNµ®œ¿ˆ©¬«÷½û¥   47.0  TN  2.0   FY11      <NA>     <NA>         <NA>   
1  TNµ®œ¿ˆ®¦¥þ¸ý¤   47.0  TN  2.0   FY11      <NA>     <NA>         <NA>   
2  TNµ®œ¿ˆ®©¯ô¸ù¤   47.0  TN  1.0   FY11      <NA>     <NA>         <NA>   
3  TNµ®œ¿ˆ®«¨ôºü    47.0  TN  2.0   FY11      <NA>     <NA>         <NA>   
4  TNµ®œ¿ˆ®«®ð¸ø¬   47.0  TN  2.0   FY11      <NA>     <NA>         <NA>   

  Responded HS_or_GED  ... Incarc Elig21 Cur

In [ ]:
print("\nCOMBINED SUMMARY:")
print(f"  Total columns: {len(combined_columns)}")
print(f"  Total 0s: {combined_columns['Count_0'].sum()}")
print(f"  Total 1s: {combined_columns['Count_1'].sum()}")
print(f"  Average % of 1s: {combined_columns['Percent_1'].mean():.2f}%")

In [ ]:
# === SAVE TO CSV ===
wave1_columns.to_csv('wave1_binary_counts.csv', index=False)
wave23_columns.to_csv('wave23_binary_counts.csv', index=False)
combined_columns.to_csv('combined_w1_w23_binary_counts.csv', index=False)

print(f"\n=== FILES SAVED ===")
print("- wave1_binary_counts.csv")
print("- wave23_binary_counts.csv") 
print("- combined_w1_w23_binary_counts.csv")


=== FILES SAVED ===
- wave1_binary_counts.csv
- wave23_binary_counts.csv
- combined_w1_w23_binary_counts.csv


In [198]:
# 1) Read each file
df1    = pd.read_csv("C:\\Users\\emros\\OneDrive\\Desktop\\NYPD\\cleaned_reports\\Processing\\bil_nytd_report_generator\\data\\wave1_binary_counts.csv")
df23   = pd.read_csv("C:\\Users\\emros\\OneDrive\\Desktop\\NYPD\\cleaned_reports\\Processing\\bil_nytd_report_generator\\data\\wave23_binary_counts.csv")
dfcomb = pd.read_csv("C:\\Users\\emros\\OneDrive\\Desktop\\NYPD\\cleaned_reports\\Processing\\bil_nytd_report_generator\\data\\combined_w1_w23_binary_counts.csv")

In [ ]:
# 2) Rename the metric columns so you can tell them apart after the join
#    (we leave "Column" as-is, and suffix everything else)
def suffix_cols(df, suffix):
    metrics = [c for c in df.columns if c != 'Column']
    return df.rename(columns={c: f"{c}_{suffix}" for c in metrics})

df1    = suffix_cols(df1,    'w1')
df23   = suffix_cols(df23,   'w23')
dfcomb = suffix_cols(dfcomb, 'combined')

In [200]:
# 3) Merge them all on "Column" with an outer join
merged = (
    df1
      .merge(df23,   on='Column', how='outer')
      .merge(dfcomb, on='Column', how='outer')
)

In [201]:
# 4) Write out the result
merged.to_csv('all_binary_counts_merged.csv', index=False)